# IRS

The IRS administers the US federal income tax system, which encompasses rates, deductions, credits, and other features.

This page shows how total federal income tax varies with employment income.
Subsections show more detail on particular programs. 

In [1]:
from fiscalsim_us import IndividualSim
import pandas as pd
import plotly.express as px

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make_tax(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members, premium_tax_credit=0)
    sim.add_household(name="household", members=members)
    sim.vary("employment_income", max=1_000_000, step=1_000)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            income_tax=sim.calc("income_tax")[0].round(),
            mtr=sim.deriv(
                "income_tax", "employment_income", wrt_target="head"
            ),
            adults=adults,
            children=str(children),
        )
    )


l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_tax(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    income_source="Income source",
    income_tax="Federal income tax",
    mtr="Marginal tax rate",
    children="Children",
    adults="Adults",
)

fig = px.line(
    df,
    "employment_income",
    "income_tax",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Federal income tax, 2022",
    color_discrete_map=COLOR_MAP,
)

fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

ModuleNotFoundError: No module named 'fiscalsim_us'

The federal income tax code creates marginal tax rates ranging from -60% to +40%, depending on the household structure and income.
The extremes at both ends are experienced by parents as their tax credits phase in and out.

In [2]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Marginal federal income tax rate",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".0%",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

NameError: name 'px' is not defined